In [39]:
import gzip 
from lxml import etree
import pandas as pd 
import xmltodict
import glob
import os 
import time 
import polars as pl 
import duckdb



In [2]:
DOSSIER_500 = "../../todo/test1000_2/"
dossier_10 = '../../todo/'

In [3]:
def timtim(fonction):
    def wrapper(*args, **kwargs):
        debut = time.time()
        resultat = fonction(*args, **kwargs)
        fin = time.time()
        temps_execution = fin - debut
        print(f"La fonction {fonction.__name__} a pris {temps_execution:.5f} secondes pour s'exécuter.")
        return resultat
    return wrapper

In [4]:
def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

In [5]:
@timtim
def Methode_1(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   liste_methode_1 = []
   for ligne_methode_1 in docbase:
     dict_temp = {}
     for clef, valolo in ligne_methode_1.items():
         if isinstance(valolo, dict) and '@V' in valolo:
             val_atrib = valolo['@V']
         else:
             val_atrib = valolo
         dict_temp[clef] = val_atrib
     liste_methode_1.append(dict_temp)
   df_methode_1 = pd.DataFrame(liste_methode_1)
   liste_df.append(df_methode_1)
  except Exception as e :
   print(f'Erreur fichier {fichier}') 
   continue 
 
 df_gros_1 = pd.concat(liste_df)
 return df_gros_1

@timtim
def Methode_2(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   df_methode_2 = pd.DataFrame(docbase)
   nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
   for col in df_methode_2.columns : 
    df_methode_2[col] = df_methode_2[col].apply(nettoyage)
   liste_df.append(df_methode_2)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue
   
 gros_df_2 = pd.concat(liste_df)
 return gros_df_2


@timtim
def Methode_3(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   df_methode_3 = pd.json_normalize(docbase)
   liste_df.append(df_methode_3)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue
 gros_df_3 = pd.concat(liste_df)
 return gros_df_3

In [6]:
dfm1 = Methode_1(DOSSIER_500)

Erreur fichier ../../todo/test1000_2\628857.xml.gz
La fonction Methode_1 a pris 31.54800 secondes pour s'exécuter.


In [7]:
dfm2 = Methode_2(DOSSIER_500)

La fonction Methode_2 a pris 21.48200 secondes pour s'exécuter.


In [8]:
dfm3 = Methode_3(DOSSIER_500)

La fonction Methode_3 a pris 19.48699 secondes pour s'exécuter.


In [20]:
gigadf1 = Methode_1(dossier_10)

Erreur fichier ../../todo\730073.xml.gz
Erreur fichier ../../todo\748147.xml.gz
Erreur fichier ../../todo\756300.xml.gz
Erreur fichier ../../todo\775775.xml.gz
Erreur fichier ../../todo\807134.xml.gz
Erreur fichier ../../todo\809162.xml.gz
Erreur fichier ../../todo\809290.xml.gz
Erreur fichier ../../todo\819789.xml.gz
Erreur fichier ../../todo\821461.xml.gz
Erreur fichier ../../todo\838663.xml.gz
La fonction Methode_1 a pris 341.34907 secondes pour s'exécuter.


In [21]:
gigadf2 = Methode_2(dossier_10)

Erreur fichier ../../todo\775775.xml.gz
La fonction Methode_2 a pris 231.86275 secondes pour s'exécuter.


In [8]:
gigadf3 = Methode_3(dossier_10)

Erreur fichier ../../todo\775775.xml.gz
La fonction Methode_3 a pris 221.79921 secondes pour s'exécuter.


Gigadf2

In [29]:
print(gigadf2.columns)
print(gigadf3.columns)

print(f'methode 2 : {gigadf2.shape} /n methode 3 : {gigadf3.shape}')

Index(['Nature', 'LibCpte', 'Fonction', 'ContNat', 'ArtSpe', 'CodRD',
       'MtBudgPrec', 'MtPropNouv', 'MtPrev', 'CredOuv', 'MtReal', 'OpBudg',
       'MtSup', 'TypOpBudg', 'MtRAR3112', 'OpeCpteTiers', 'CaracSup',
       'MtRARPrec', 'ContOp', 'Operation', 'ContFon'],
      dtype='object')
Index(['Nature.@V', 'LibCpte.@V', 'Fonction.@V', 'ContNat.@V', 'ArtSpe.@V',
       'CodRD.@V', 'MtBudgPrec.@V', 'MtPropNouv.@V', 'MtPrev.@V', 'CredOuv.@V',
       'MtReal.@V', 'OpBudg.@V', 'MtSup.@Code', 'MtSup.@V', 'MtSup',
       'TypOpBudg.@V', 'MtRAR3112.@V', 'OpeCpteTiers.@V', 'CaracSup.@Code',
       'CaracSup.@V', 'MtRARPrec.@V', 'ContOp.@V', 'CaracSup', 'Operation.@V',
       'ContFon.@V'],
      dtype='object')
methode 2 : (1429917, 21) /n methode 3 : (1429912, 25)


Gigadf3

Methode Polars

In [ ]:
fichier_test = './771014.xml.gz'
test_parse = parse_fichier(fichier_test)
dict_parse = test_parse['DocumentBudgetaire']['Budget']['LigneBudget']
dict_parse

In [6]:

def extract_value(d):
    if isinstance(d, dict) and '@V' in d:
        return d['@V']
    return d
   
SCHEMA = {"Nature" : pl.Object,
          "LibCpte" : pl.Object,
          "Fonction" : pl.Object,
          "Operation" : pl.Object,
          "ContNat" : pl.Object,
          "ArtSpe" : pl.Object,
          "ContFon" : pl.Object,
          "ContOp" : pl.Object,
          "CodRD" : pl.Object,
          "MtBudgPrec" : pl.Object,
          "MtRARPrec" : pl.Object,
          "MtPropNouv" : pl.Object,
          "MtPrev" : pl.Object,
          "CredOuv" : pl.Object,
          "MtReal" : pl.Object,
          "MtRAR3112" : pl.Object,
          "OpBudg" : pl.Object,
          "TypOpBudg" : pl.Object,
          "OpeCpteTiers" : pl.Object, 
          "MtSup" : pl.Object, 
          "CaracSup" : pl.Object
          }

#liste_test = []
#for ssr in dict_parse : 
# dict_temp = {key: extract_value(value) for key, value in ssr.items()}
# liste_test.append(dict_temp)

#liste_test[1]

In [36]:
pl_test = pl.DataFrame(liste_test, schema=schema)
pl_test.head(2)

Nature,LibCpte,Fonction,Operation,ContNat,ArtSpe,ContFon,ContOp,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,TypOpBudg,OpeCpteTiers,MtSup,CaracSup
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
001,None,01,None,001,false,None,None,D,0.00,0.00,0.00,0.00,160523368.02,160523368.02,0.00,0,None,None,"[{'@Code': 'ProdChaRat', '@V': '0.00'}, {'@Code': 'BudgetHorsRAR', '@V': '160523368.02'}]",None
002,None,01,None,002,false,None,None,R,0.00,0.00,0.00,0.00,27137951.23,27137951.23,0.00,0,None,None,"[{'@Code': 'ProdChaRat', '@V': '0.00'}, {'@Code': 'BudgetHorsRAR', '@V': '27137951.23'}]",None


In [7]:
@timtim
def Methode_polars(chemin, schema) : 
 liste_prepl = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   pre_df = pl.DataFrame(docbase, schema=schema)
   liste_prepl.append(pre_df)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue
 pl_test = pl.concat(liste_prepl)
 return pl_test 

In [8]:
comparatif_pl = Methode_polars(dossier_10, schema=SCHEMA)

Erreur fichier ../../todo\730073.xml.gz
Erreur fichier ../../todo\748147.xml.gz
Erreur fichier ../../todo\756300.xml.gz
Erreur fichier ../../todo\775775.xml.gz
Erreur fichier ../../todo\807134.xml.gz
Erreur fichier ../../todo\809162.xml.gz
Erreur fichier ../../todo\809290.xml.gz
Erreur fichier ../../todo\819789.xml.gz
Erreur fichier ../../todo\821461.xml.gz
Erreur fichier ../../todo\838663.xml.gz
La fonction Methode_polars a pris 257.72901 secondes pour s'exécuter.


In [9]:
comparatif_meth2 = Methode_2(dossier_10)

Erreur fichier ../../todo\775775.xml.gz
La fonction Methode_2 a pris 244.25491 secondes pour s'exécuter.


Erreurs methode_1 : 

- Erreur fichier ../../todo\730073.xml.gz
- Erreur fichier ../../todo\748147.xml.gz
- Erreur fichier ../../todo\756300.xml.gz
- Erreur fichier ../../todo\775775.xml.gz
- Erreur fichier ../../todo\807134.xml.gz
- Erreur fichier ../../todo\809162.xml.gz
- Erreur fichier ../../todo\809290.xml.gz
- Erreur fichier ../../todo\819789.xml.gz
- Erreur fichier ../../todo\821461.xml.gz
- Erreur fichier ../../todo\838663.xml.gz
- La fonction Methode_1 a pris 341.34907 secondes pour s'exécuter.


In [29]:
fichier_test = './771014.xml.gz'
test_parse = parse_fichier(fichier_test)
dict_parse = test_parse['DocumentBudgetaire']['Budget']['LigneBudget']
dict_meta = test_parse['DocumentBudgetaire']

Ajout de la verif sur le fichier parquet 

1. Methode Duckdb 

In [ ]:
from minio import Minio
import duckdb

client = Minio(
    endpoint='localhost:9000',
    access_key='airflow-local',
    secret_key='airflow-local',
    secure = False
  )

In [11]:
def recherche_id_fichier() :
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet('./parquet-proto-doc-budget')
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id

In [26]:
liste_ttt = []
for i in glob.glob(os.path.join(dossier_10, "*.gz")) : 
 liste_ttt.append(i)

liste_ttt[7]

'../../todo\\20201113-1875908.xml.gz'

In [28]:
fichier_test = '../../todo/727285.xml.gz'
id_test = _isolement_id(liste_ttt[1])
id_test

'1825601'

In [32]:
dict_meta = test_parse['DocumentBudgetaire']['EnTeteDocBudgetaire']
dict_meta

{'DteStr': {'@V': '2021-06-02'},
 'LibellePoste': {'@V': 'le Trésorier de la MEL'},
 'IdPost': {'@V': '059053'},
 'LibelleColl': {'@V': 'METROPOLE EUROPEENNE DE LILLE'},
 'IdColl': {'@V': '20009320100016'},
 'NatCEPL': {'@V': '06 Groupements de collectivités'}}

In [ ]:
dict_meta['Id_Fichier'] = {'@V' : id_test}
dict_meta

dict_glob = dict_parse
for i in dict_glob :
    i.update(dict_meta)
dict_glob

In [42]:
def recherche_id_fichier(chemin_parquet) :
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet(chemin_parquet)
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id


def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité'''
 val_id_fichier_source = fichier.split("\\")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier


@timtim
def Methode_2(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   df_methode_2 = pd.DataFrame(docbase)
   nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
   for col in df_methode_2.columns : 
    df_methode_2[col] = df_methode_2[col].apply(nettoyage)
   liste_df.append(df_methode_2)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue
   
 gros_df_2 = pd.concat(liste_df)
 return gros_df_2

In [50]:
@timtim
def Methode_2_1(chemin, chemin_verif) : 
 liste_id = recherche_id_fichier(chemin_verif)
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  id_fichier = _isolement_id(fichier)
  #Fichier non pre existant dans la db / parquet
  if id_fichier in liste_id :
   pass 
  else : 
   try : 
     fichier_parse = parse_fichier(fichier)
     metadonnees = fichier_parse['DocumentBudgetaire']['EnTeteDocBudgetaire']
     donnes_budget = fichier_parse['DocumentBudgetaire']['Budget']['LigneBudget']
     metadonnees['Id_Fichier'] = {'@V' : id_fichier}
     for i in donnes_budget : 
      i.update(metadonnees)
     def_2_1 = pd.DataFrame(donnes_budget)
     nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
     for col in def_2_1.columns : 
      def_2_1[col] = def_2_1[col].apply(nettoyage)
     liste_df.append(def_2_1)
   except Exception as e : 
     print(f'Erreur fichier {fichier}')
     continue
 df_2_1 = pd.concat(liste_df)
 df_2_1.to_parquet('parquet_renouveau', compression= 'gzip')
 return df_2_1 


In [52]:
chemin_origin_parquet = './parquet-proto-doc-budget'
chemin_parquet_verif = './parquet_renouveau'


Version sans la verif, permet de créer le fichier parquet 

In [49]:
df_2_1 = Methode_2_1(dossier_10, chemin_parquet_verif)

Erreur fichier ../../todo\730073.xml.gz
Erreur fichier ../../todo\748147.xml.gz
Erreur fichier ../../todo\756300.xml.gz
Erreur fichier ../../todo\775775.xml.gz
Erreur fichier ../../todo\807134.xml.gz
Erreur fichier ../../todo\809162.xml.gz
Erreur fichier ../../todo\809290.xml.gz
Erreur fichier ../../todo\819789.xml.gz
Erreur fichier ../../todo\821461.xml.gz
Erreur fichier ../../todo\838663.xml.gz
La fonction Methode_2_1 a pris 330.14720 secondes pour s'exécuter.


In [53]:
df_2_1 = Methode_2_1(DOSSIER_500, chemin_parquet_verif)

Erreur fichier ../../todo/test1000_2\628857.xml.gz
La fonction Methode_2_1 a pris 41.37900 secondes pour s'exécuter.
